# Segmenting and Clustering Neighborhoods in Toronto

## Part 1: Create the dataframe

Credit: [Web scraping HTML tables with python](https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059)

In [153]:
# pip install lxml # comment if lxml is not already installed

In [154]:
import requests
import lxml.html as lh
import pandas as pd

In [155]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [156]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [157]:
#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('%d:"%s"'%(i,name)) 
    col.append((name,[]))
# %s acts a placeholder for a string while %d acts as a placeholder for a number. Their associated values are passed in via a tuple using the % operator.

1:"Postal code
"
2:"Borough
"
3:"Neighborhood
"


In [158]:
col

[('Postal code\n', []), ('Borough\n', []), ('Neighborhood\n', [])]

There is an additional "\n" at the end of the strings. We need to remove it.

In [159]:
#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    if name[-1] == '\n':
        name = name[:-1]
    print('%d:"%s"'%(i,name)) 
    col.append((name,[]))

1:"Postal code"
2:"Borough"
3:"Neighborhood"


In [160]:
col

[('Postal code', []), ('Borough', []), ('Neighborhood', [])]

Looks good now! Now we can write the data into the list col

In [161]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content()
        # drop the \n at the end
        if data[-1] == '\n':
            data = data[:-1]       
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

Convert it to a dataframe

In [162]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [163]:
df

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...
179,M9Z,Not assigned,


Last row is wrong and needs to be removed.

In [164]:
df0 = df.iloc[:-1]

In [165]:
df0

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


In [166]:
df0.shape

(180, 3)

Remove rows with a Borough that is not assigned

In [167]:
df1 = df0[df0['Borough'] != 'Not assigned']
df1.reset_index(drop = True, inplace = True)
df1.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [168]:
df1.shape

(103, 3)

Put Neighborhoods with the same postal code in one row but separate them with a comma

In [169]:
df2 = df1.apply(lambda x: x.str.replace(' /',','))
df2.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [170]:
# check if there is a cell with not assigned neighborhood
df2['Neighborhood'].isnull().unique()

array([False])

This means all cells in Neighborhood has already been assigned. 

In [171]:
df2.shape

(103, 3)

## Part 2 Adding latitute and longitude

In [174]:
# pip install geocoder # uncomment if geocoder is not already installed

In [175]:
# import geocoder
# g = geocoder.google('Mountain View, CA')
# g.latlng

In [176]:
# initialize your variable to None
# lat_lng_coords = None

# loop until you get the coordinates
# while(lat_lng_coords is None):
  # g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  # lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

Tried geocoder, didn't work. Let's use the CSV file instead.

In [177]:
csv_path = 'http://cocl.us/Geospatial_data'

In [178]:
location = pd.read_csv(csv_path)
print ('Data read into a pandas dataframe!')

Data read into a pandas dataframe!


In [179]:
location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now let's merge the two dataframes. 
The column names for 'Postal Code' is slightly different in the two dataframes. Match them first. 

In [180]:
df2 = df2.rename(columns={'Postal code': 'Postal Code'})
df2.head(3)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [181]:
df3 = df2.join(location.set_index('Postal Code'), on='Postal Code')
df3.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [182]:
df3.shape

(103, 5)

## Part 3 Clustering

In [ ]:
import numpy as np

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: \ 

First we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [ ]:
# Define API credential
CLIENT_ID = 'S1KPK21EQRSXK5MGH5W3CLNRJAU0KO2NHHW2C3RISMQLZEZP' # your Foursquare ID
CLIENT_SECRET = '2KXEE03BBQSBTR20AIOOHIIYTNQBBKMR1KT0IKQFWDJD4N2P' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

Let's create a function that extracts the category of the venue.

In [ ]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now let's find boroughs that contain the word Toronto and put the data in a new dataframe.

In [ ]:
df4 = df3[df3['Borough'].str.contains('Toronto')]
df4.reset_index(drop = True, inplace = True)
df4.shape

In [ ]:
df4.head()

We can run getNearbyVenues function now to get the data.

In [ ]:
Toronto_venues = getNearbyVenues(names=df4['Neighborhood'],
                                   latitudes=df4['Latitude'],
                                   longitudes=df4['Longitude'])

In [ ]:
print(Toronto_venues.shape)
Toronto_venues.head()

Use groupby() to check how many venues were found for each neighborhood or neighhood group (under same Postal Code)

In [ ]:
Toronto_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

In [ ]:
'Neighborhood' in Toronto_venues['Venue Category'].unique()

An interesting thing is that one of the Venue Category is called Neighborhood. We will change our previous 'Neighborhood' column to 'Neighborhood Name'

#### Explore each neighborhood/neighborhood group

In [ ]:
# one hot encoding
T_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
T_onehot.head(3)

In [ ]:
T_onehot.shape

In [ ]:
# add neighborhood name column back to T_onehot dataframe as the first column
T_onehot.insert(0, 'Neighborhood Name', Toronto_venues['Neighborhood'])

T_onehot.head()

In [ ]:
T_onehot.shape

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
T_grouped = T_onehot.groupby('Neighborhood Name').mean().reset_index()
T_grouped.head()

In [ ]:
T_grouped.shape

Now let's create a new dataframe and display the top 10 venues for each neighborhood

To achieve that, we need to define a function to sort the venues in descending order

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues] # only return the names not the freq.

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = T_grouped['Neighborhood Name']

for ind in np.arange(T_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(T_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
neighborhoods_venues_sorted.shape

#### Clustering the neighborhoods

In [ ]:
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

T_grouped_clustering = T_grouped.drop('Neighborhood Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(T_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

T_merged = df4
T_merged = T_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

T_merged.head()

#### Finally, visualization

Use geopy library to get the latitude and longitude values of Toronto.

In [ ]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Toronto using latitude and longitude values
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [ ]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(T_merged['Latitude'], T_merged['Longitude'], T_merged['Neighborhood'], T_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine each cluster

We can take a look at each cluster for their featured venues.

In [ ]:
# Select what columns to show
col = T_merged.columns[[2] + list(range(6, T_merged.shape[1]))]
col

Cluster 0

In [ ]:
Cluster_0 = T_merged.loc[T_merged['Cluster Labels'] == 0, col]
Cluster_0.head()

In [ ]:
print ('There are {} neighborhood groups in Cluster 0'.format(Cluster_0.shape[0]))

Cluster 1

In [ ]:
Cluster_1 = T_merged.loc[T_merged['Cluster Labels'] == 1, col]
Cluster_1.head()

Cluster 2

In [ ]:
Cluster_2 = T_merged.loc[T_merged['Cluster Labels'] == 2, col]
Cluster_2.head()

Cluster 3

In [ ]:
Cluster_3 = T_merged.loc[T_merged['Cluster Labels'] == 3, col]
Cluster_3.head()

Cluster 4

In [ ]:
Cluster_4 = T_merged.loc[T_merged['Cluster Labels'] == 4, col]
Cluster_4.head()